In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
from torchvision.transforms import ToTensor

# Image Processessing

In [3]:
def resize_images_in_dir(raw_data_path:Path, width:int, height:int):
	for img_path in raw_data_path.iterdir():
		img = Image.open(img_path)
		img = img.resize((width, height), Image.ANTIALIAS)
		img.save(img_path)

In [4]:
class FaceClassifierDataset(Dataset):
    def __init__(self, annotations_file, img_dir):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = self.img_dir / str(self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        image = self.transform(image)
        label = self.img_labels.iloc[idx, 1]
        label = self.target_transform(label)
        return image, label

    def transform(self, image):
        return ToTensor(image)

    def target_transform(self, label):
        if label.upper() == "TAYLOR":
            return 0
        elif label.upper() == "ROCK":
            return 1
        elif label.upper() == "QUENTIN":
            return 2

In [5]:
root_data_path = Path("/home/hpc/kurlanl1/CSC-380/CSC380-Artificial-Intelligence/faces_dataset")
train_data_path = root_data_path / "train"
test_data_path = root_data_path / "train"

train_dataset = FaceClassifierDataset(train_data_path / "y" / "y.csv", 
					test_data_path / "X")
test_dataset = FaceClassifierDataset(test_data_path / "y" / "y.csv", 
					test_data_path / "X")

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)